<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="../image/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第二章:处理文本数据 

本章节需要安装的包

pip3 install importlib.metadata

In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
# 确认库已安装并显示当前安装的版本

torch version: 2.10.0
tiktoken version: 0.12.0


- 本章节已经为LLM的实现构建了数据集

<img src="../image/01.webp" width="500px">

## 2.1 理解文字embedding

- 无代码

- 在众多形式的embedding中,我们只讨论text embedding
- embedding含义丰富,而且是常用词汇,所以以下皆不做翻译,多加体会!

<img src="../image/02.webp" width="500px">

- LLM从高纬空间视角理解文字(i.e., 上千个dimension)
- 虽然人类只能想象低维的视角,我们无法描绘计算机所理解的embedding
- 但是下图我们粗浅的从二维上模拟计算机的视角

<img src="../image/03.webp" width="300px">

## 2.2 文本标签化(tokenize)

- 关于embedding token,实在是不好翻译,于是有时候我会选择不去翻译这两个专有名词
- 本节中,我们将tokenize文字信息. 这会把文字拆解为更多小的理解单元 例如单个词或者字节

(这也有点抽象，事实上可以粗略理解为将单词拆分为词根、词源和词缀)

<img src="../image/04.webp" width="300px">

- 载入源文件
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) 一本无版权的短篇小说

In [3]:
import os##导入os库    
import urllib.request ##导入request库

if not os.path.exists("the-verdict.txt"):##如果文件不存在则创建，防止因文件已存在而报错
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)##从指定的地点读取文件

- 如果在执行前面的代码单元时遇到 `ssl.SSLCertVerificationError` 错误可能是由于使用了过时的 Python 版本；
- 你可以在 [GitHub 上查阅更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)。

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read() ##读入文件按照utf-8
    
print("Total number of character:", len(raw_text))##先输出总长度
print(raw_text[:99])##输出前一百个内容

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对这段文本进行分词和嵌入处理，以便用于大语言模型（LLM）。
- 我们将基于一些简单的示例文本开发一个简单的分词器，之后可以将其应用于上述文本。
- 以下正则表达式将基于空格进行分割。

In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)##正则表达式按照空白字符进行分割

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 优化正则表达,可以分割更多的符号

In [6]:
result = re.split(r'([,.]|\s)', text)##只是按照, .分割
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 移除空格

In [7]:
##把上述结果去掉空格
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 我们还需要处理其他标点符号

In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) ##就是按照常用的符号分割
result = [item.strip() for item in result if item.strip()]##去掉两端的空白字符 也是去掉了空字符串与仅包含空白字符的项
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 万事俱备，我们现在来看一下文字处理后的效果

<img src="../image/05.webp" width="350px">

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) ##按照符号继续把原文件给分割了
preprocessed = [item.strip() for item in preprocessed if item.strip()]##去掉两端的空白字符 也是去掉了空字符串和仅包含空白字符的项
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看token的长度

In [10]:
print(len(preprocessed))

4690


## 2.3 给token编号

- 通过如下的embedding层,我们可以给token编号

<img src="../image/06.webp" width="500px">

- 我们要创建一个表格,给所有的token给映射到不同的标号上

In [11]:
all_words = sorted(set(preprocessed))#从去掉重复的字符
vocab_size = len(all_words)#计总的单词书

print(vocab_size)

1130


In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}##先把word进行编号,再按照单词或者标点为索引(有HashList那味道了)

- 看一下前50个是怎样的

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break ##遍历到前五十个

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 接下来,我们将通过一个短文本来感受下,处理后的效果是怎样的

<img src="../image/07.webp" width="500px">

- 现在将所有内容整合到一个分词器类中。

In [14]:
class SimpleTokenizerV1:#一个实例的名字创立
    def __init__(self, vocab): ## 初始化一个字符串
        self.str_to_int = vocab #单词到整数的映射
        self.int_to_str = {i:s for s,i in vocab.items()} 
        #方便解码,进行整数到词汇的反向映射
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)##正则化分词标点符号
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()## 去掉两端空格与全部的空句
        ]
        ids = [self.str_to_int[s] for s in preprocessed]##整理完的额字符串列表对应到id,从字典出来
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) #映射整数id到字符串。join是用前面那个(“ ”)联结成一个完整的字符串
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #使用正则表达式，去除标点符号前的多余空格
        # \s+匹配一个或者多个空白  \1 替换到匹配
        return text

- `encode` 函数将文本转换为token ID。
- `decode` 函数将token ID 转换回文本。

<img src="../image/08.webp" width="500px">

- 我们可以使用分词器将文本编码（即分词）为数字。
- 然后，这些整数可以作为大语言模型（LLM）的输入，进行嵌入。

In [15]:
tokenizer = SimpleTokenizerV1(vocab) #用vocab创造一个实例

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text) #按照这个例子里的encode函数处理text
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 把数字重新映射回文字

In [16]:
tokenizer.decode(ids)#按照这个例子里的decode函数处理text

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
tokenizer.decode(tokenizer.encode(text))#按照这个例子里的decode函数处理(#按照这个例子里的encode函数处理text)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊token

- 文本的结尾需要特别的符号来表明

<img src="../image/09.webp" width="500px">

- 一些分词器使用特殊token来帮助大语言模型（LLM）获取额外的上下文信息。
- 其中一些特殊token包括：
  - `[BOS]`（序列开始）表示文本的开始。
  - `[EOS]`（序列结束）表示文本的结束（通常用于连接多个不相关的文本，例如两个不同的维基百科文章或两本不同的书籍等）。
  - `[PAD]`（填充）如果我们使用大于1的批次大小训练LLM（我们可能会包含不同长度的多篇文本），使用填充token将较短的文本填充至最长的长度，以确保所有文本具有相同的长度。
- `[UNK]` 用于表示词汇表中没有的词。

- 请注意，GPT-2不需要上述提到的这些token，它只使用 `<|endoftext|>` token。
- `<|endoftext|>` 类似于上述提到的 `[EOS]` token。
- GPT 还使用 `<|endoftext|>` 进行填充（因为我们在批量输入训练时通常使用掩码，所以无论这些填充token是什么，都不会影响模型的训练，因为填充token不会被关注）。
- GPT-2 不使用 `<UNK>` token来表示词汇表外的词；相反，GPT-2 使用字节对编码（BPE）分词器，将单词分解为子词单元，后续将进一步讨论这一点。
- 我们在两个独立文本之间使用 `<|endoftext|>` token：

<img src="../image/10.webp" width="500px">

- 看一下接下来会发生什么

In [19]:
tokenizer = SimpleTokenizerV1(vocab)  ##用vocab创造一个实例

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上述操作会产生一个错误，因为单词“Hello”不在词汇表中。
- 为了处理这种情况，我们可以向词汇表中添加类似 `"<|unk|>"` 的特殊token，用于表示未知词汇。
- 因为我们已经在扩展词汇表，那么我们可以再添加一个token `"<|endoftext|>"`，该token在GPT-2训练中用于表示文本的结束（它也用于连接的文本之间，例如当我们的训练数据集包含多篇文章、书籍等时）。

In [20]:
all_tokens = sorted(list(set(preprocessed)))#set去重 list把处理后的重新变为列表,然后排序
all_tokens.extend(["<|endoftext|>", "<|unk|>"])#加上未知的表示

vocab = {token:integer for integer,token in enumerate(all_tokens)}
#遍历 enumerate(all_tokens) 中的每个元组 (integer, token)，以 token 作为键，integer 作为值创建字典条目。

In [21]:
len(vocab.items())

1132

In [22]:
for i, item in enumerate(list(vocab.items())[-5:]):#输出后五个内容与其标号
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 因此增加 `<unk>`不失为一种好的选择

In [23]:
class SimpleTokenizerV2:##版本2.0,启动!
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}#s为单词,i是key
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)#正则化按照标点分类
        preprocessed = [item.strip() for item in preprocessed if item.strip()]#去掉两头与所有空余句
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
            #遍历 preprocessed 中的每个 item，如果 item 存在于 self.str_to_int（即词汇表）中，就保留 item
            #如果不存在（即该单词或符号未定义在词汇表中），就替换为特殊token <|unk|>。
            #拓展:推导式（如列表推导式）是一种紧凑的语法，专门用于生成新列表（或其他容器）
            #与普通 for 循环相比，它更加简洁和高效，但逻辑复杂时可能会降低可读性。
        ]

        ids = [self.str_to_int[s] for s in preprocessed]#单词或标点映射为整数列表
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

- 用优化后的分词器对文本进行操作

In [24]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))#用句子分隔符链接两个句子

print(text) #跟第一个一样,但不会报错了

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [25]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [26]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2 使用字节对编码（BPE）作为其分词器。
- 这种方式允许模型将不在预定义词汇表中的单词分解为更小的子词单元，甚至是单个字符，从而使其能够处理词汇表外的词汇。
- 例如，如果 GPT-2 的词汇表中没有“unfamiliarword”这个单词，它可能会将其分词为 ["unfam", "iliar", "word"]，或者根据训练好的 BPE 合并规则进行其他子词分解。
- 原始的 BPE 分词器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用了 OpenAI 开源的 [tiktoken](https://github.com/openai/tiktoken) 库中的 BPE 分词器，该库在 Rust 中实现了核心算法，以提高计算性能。
- 在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，对比了这两种实现的效果（在样本文本上，tiktoken 的速度大约快了 5 倍）。

In [27]:
# pip install tiktoken

In [28]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))#验证下载并输出版本信息

tiktoken version: 0.12.0


In [29]:
tokenizer = tiktoken.get_encoding("gpt2")#初始化GPT2!

In [30]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})#输出分词的id,可以允许endoftext

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [31]:
strings = tokenizer.decode(integers)
#按照数字解码回去

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers将未知词汇分解为子词和单个字符。

<img src="../image/11.webp" width="300px">

## 2.6 利用滑动窗口进行数据采样

- 现在我们训练的大语言模型（LLMs）时是一次生成一个单词，因此希望根据训练数据的要求进行准备，使得序列中的下一个单词作为预测目标。

<img src="../image/12.webp" width="400px">

In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)#读入了一个text并编码到enc_text里面
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标。
- 由于我们希望模型预测下一个单词，因此我们要生成目标是将输入右移一个位置后的单词。

In [33]:
enc_sample = enc_text[50:]#从第五十一个开始向后

In [34]:
context_size = 4 #sliding windows4

x = enc_sample[:context_size]#开头四个
y = enc_sample[1:context_size+1]#第二个开始的四个

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 就像预言家一个晚上只能预言一个玩家,我们的模型一次也只能预测一个单词

In [35]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么编号
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [36]:
for i in range(1, context_size+1):
    #文本成输入 context,先输出有什么,然后输出下一个是什么单词
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在后续章节中处理下一个单词预测，届时会介绍注意力机制。
- 目前，我们实现一个简单的数据加载器，它遍历输入数据集并返回右移一个位置后的输入和目标。

- 安装并导入 PyTorch（安装提示请参见附录A）。

In [37]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.10.0


- 用滑动窗口法运行,窗口位置每次加一

<img src="../image/13.webp" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取文本块。

In [38]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    """
    自定义数据集类，用于将输入文本按滑动窗口切割成GPT训练用的样本对。
    每个样本包含：
      - 输入序列（input_ids）：长度为 max_length 的token列表
      - 目标序列（target_ids）：长度同上，对应input_ids的右移一位版本（即“下一个词”）
    """

    def __init__(self, txt, tokenizer, max_length, stride):
        """
        参数说明:
            txt:       完整文本字符串，作为训练语料
            tokenizer: 分词器（应支持 encode 方法，将文本转为id序列）
            max_length: 每个输入序列的最大长度（上下文窗口长度）
            stride:    滑窗步长（窗口每次向后滑动的token数，stride < max_length会有重叠）
        """
        self.input_ids = []   # 存储输入序列（每个元素为长度max_length的1D tensor）
        self.target_ids = []  # 存储目标序列（与input_ids一一对应）

        # 1. 将完整文本序列化为token id序列
        # allowed_special={"<|endoftext|>"} 允许特殊符号 "<|endoftext|>" 出现在编码中
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})  # 例如 [15496, 11, 318, ...]

        # 2. 用滑动窗口法切分数据（得到有重叠的输入-目标对）
        # 例如: max_length=4, stride=2, token_ids=[1,2,3,4,5,6 ...]
        #   第1个样本: input:[1,2,3,4], target:[2,3,4,5]
        #   第2个样本: input:[3,4,5,6], target:[4,5,6,7] （for i in range(0, ..., stride)）
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]                   # 当前样本输入
            target_chunk = token_ids[i + 1 : i + max_length + 1]          # 当前样本对应的“下一个词”
            self.input_ids.append(torch.tensor(input_chunk))              # 转为tensor存储
            self.target_ids.append(torch.tensor(target_chunk))

        # 注意: 一共会生成  floor((len(token_ids) - max_length) / stride) 个样本

    def __len__(self):
        """
        返回样本数量，即所有滑窗所得的输入序列个数。
        """
        return len(self.input_ids)

    def __getitem__(self, idx):
        """
        按索引返回(input_ids, target_ids)的tensor对。
        用于dataloader按批次取样本。
        """
        return self.input_ids[idx], self.target_ids[idx]

In [39]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    根据提供的文本txt，创建一个适用于GPT风格小模型训练的数据加载器（DataLoader）。
    支持分批次（batch）、滑动窗口切分、数据混洗等功能。

    参数说明:
        txt:           完整的文本字符串，将作为训练语料使用
        batch_size:    每个批次包含的样本数
        max_length:    每个输入序列的最大长度（即上下文窗口大小）
        stride:        每次滑窗向后移动的token数（stride < max_length表示有重叠）
        shuffle:       是否对数据集中的样本顺序进行混洗（通常训练要True，验证/测试为False）
        drop_last:     如果最后一个批次不足batch_size，是否丢弃（True只保留完整批次）
        num_workers:   多少个子进程加载数据，0为主进程（小数据为0即可）

    返回:
        返回一个PyTorch的DataLoader实例，可以迭代得到(batch_size, max_length)大小的张量对。
    """

    # 1. 初始化分词器（Tokenizer）
    # 这里使用tiktoken库提供的GPT2模型分词器，将原始文本拆分为token（数字id），
    # 后续输入到模型中。tiktoken.get_encoding("gpt2")会返回一个“编码对象”。
    tokenizer = tiktoken.get_encoding("gpt2")

    # 2. 构建自定义数据集对象GPTDatasetV1
    # 这个数据集会将文本先编码为token id序列，再按滑动窗口切成一对对的
    # (input_ids, target_ids)。这样每一条样本都是模型一次要“看”的上下文窗口。
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    # dataset长度约等于 floor((len(token_id序列) - max_length) / stride)
    # 每条样本都是两组长度max_length的id列表

    # 3. 构建PyTorch原生的DataLoader
    # 用于批量加载dataset里的样本，并支持shuffle、drop_last、并行读取等功能。
    dataloader = DataLoader(
        dataset,                  # 数据集对象，__getitem__返回(input_ids, target_ids)
        batch_size=batch_size,    # 批次大小，即每次返回多少个样本组成一批
        shuffle=shuffle,          # 是否混洗样本（训练常开，推理常关）
        drop_last=drop_last,      # 若最后一批不足量是否丢弃
        num_workers=num_workers   # 子进程数量（文本小可为0；大数据集可加速读取）
    )

    # 4. 返回构建好的dataloader对象
    # 用户可以像for batch in dataloader: ... 这样遍历每个批次。
    return dataloader

- 让我们使用批次大小为1、上下文大小为4的设置，测试数据加载器在大语言模型（LLM）中的表现。

In [40]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
# 创建一个dataloader对象，负责将原始文本raw_text按批次（batch_size=1）、上下文窗口大小（max_length=4）、滑窗步幅（stride=1）分割成训练样本，并且不打乱顺序 (shuffle=False)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

# 将dataloader转换为迭代器，用于逐批次遍历样本
data_iter = iter(dataloader)

# 取出第一个批次的数据（每个batch是一个(input_ids, target_ids)的元组，都是长度为max_length的token id张量）
first_batch = next(data_iter)

print(first_batch)  # 输出第一个批次的内容，便于检查分割效果

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [42]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 下面是一个示例，步幅等于上下文长度（此处为4）：

<img src="../image/14.webp" width="500px">

- 我们还可以批量输出。
- 因为过多的重叠可能导致过拟合,这里增加了步幅。

In [ ]:
# 使用 create_dataloader_v1 创建一个数据加载器对象 dataloader
# - raw_text: 原始文本字符串，将作为数据集来源
# - batch_size=8: 每次迭代返回8个样本组成一个批次
# - max_length=4: 每个样本的输入和目标序列的长度（token数量）为4
# - stride=4: 窗口滑动步幅为4，即每获取一个样本后，将窗口向右移动4个token，下一个样本不会与前一个样本重叠
# - shuffle=False: 不打乱样本顺序，按文本顺序生成
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_length=4,
    stride=4,
    shuffle=False
)

# 将 dataloader 转为迭代器以便用 next() 按批次获取数据
data_iter = iter(dataloader)

# 获取数据加载器的下一个批次
# - 返回一个二元组(inputs, targets)，它们都是 shape=(batch_size, max_length) 的张量
#   - inputs: 当前批次的输入token id序列
#   - targets: 当前批次的目标token id序列（一般是inputs右移一位的下一个token）
inputs, targets = next(data_iter)

# 打印本批次的输入张量
print("Inputs:\n", inputs)

# 打印本批次的目标张量
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建token嵌入(Creating token embeddings)

- 数据集已准备好用于大语言模型（LLM）。
- 最后，让我们使用嵌入层将token转换为连续的向量表示。
- 通常，这些嵌入层是大语言模型的一部分，并在模型训练过程中进行更新（训练）。

<img src="../image/15.webp" width="400px">

- 假设我们有以下四个输入示例，经过分词后对应的输入 ID 为 2、3、5 和 1：

In [44]:
input_ids = torch.tensor([2, 3, 5, 1])#要加入2,3,5,1的字符

- 为了简化问题，假设我们只有一个包含 6 个词的小型词汇表，并且我们希望创建大小为 3 的嵌入。

In [ ]:
# 指定嵌入层支持的唯一token（词汇表）总数，这里我们假设有6个不同的token（编号为0 ~ 5）
vocab_size = 6  # 例如 ["cat", "dog", "apple", "hello", "world", "."] 这样的词汇表有6个唯一词

# 指定嵌入向量的维度，也就是每个token将被映射为一个3维向量
output_dim = 3  # 每个token经过嵌入层后会变成一个长度为3的实数向量

# 设置随机种子，保证每次运行输出的结果（如权重初始化）一致，便于调试和复现
torch.manual_seed(123)

# 创建一个嵌入层对象：
# - 输入参数 vocab_size 表示一共要学习多少个token的表示
# - 输入参数 output_dim 表示每个token的表示有多少维
# - 嵌入层可以看作一个学习得到的查找表，其内部参数是一个形状为 (vocab_size, output_dim) 的权重矩阵
# - 其中第i行就是第i号token对应的向量表示
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)  # 每一行对应一个token的嵌入向量

- 结果是个6*3的矩阵

In [46]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉（one-hot encoding）的人来说，上述嵌入层方法本质上只是实现one-hot编码后接矩阵乘法的更高效方式，具体实现可以参考[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)中的补充代码。
- 嵌入层只是对one-hot encoding和矩阵乘法方法的高效实现，因此可以将其视为一个神经网络层，并通过反向传播进行优化。

- 通过下列操作,我们可以吧id3映射到一个三纬的矩阵

In [47]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上述内容是 `embedding_layer` 权重矩阵中的第四行。
- 要嵌入上述所有四个 `input_ids` 值，我们执行以下操作：

In [49]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="../image/16.webp" width="500px">

- **您可能对比较嵌入层与常规线性层的附加内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 位置信息编码

- 嵌入层将 ID 转换为相同的向量表示

<img src="../image/17.webp" width="400px">

- 位置信息与token向量结合，形成大语言模型的最终输入：

<img src="../image/18.webp" width="500px">

- 字节对编码器的词汇表大小为 50,257：
- 假设我们想将输入token编码为一个 256 维的向量表示：

In [53]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)#映射为tensor

- 如果我们从数据加载器中采样数据，我们将每个批次中的token嵌入为一个 256 维的向量。
- 如果我们有一个批次大小为 8，每个批次包含 4 个token，这将得到一个 8 x 4 x 256 的张量：

In [54]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [55]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [56]:
token_embeddings = token_embedding_layer(inputs)#调用token_embedding_layer将输入inputs映射为对应的嵌入向量。
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需要创建另一个位置嵌入层：

In [57]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
#目的是为输入序列中的每个位置生成一个向量,表明位置信息

- 嵌入层本质上是一个查找表,大小为(context_length, output_dim)

In [59]:
# 先生成 [0, 1, ..., max_length-1]，每个位置用一个整数ID表示
position_ids = torch.arange(max_length)
# 再通过位置嵌入层按ID查表：每个位置ID对应output_dim维的位置向量
pos_embeddings = pos_embedding_layer(position_ids)
# 打印位置嵌入张量形状，常见为 (max_length, output_dim)

print(pos_embeddings.shape)

torch.Size([4, 256])


In [60]:
print(pos_embeddings)

tensor([[-1.4693,  1.0024,  0.6403,  ..., -0.7098, -0.4741,  1.3287],
        [-0.3833,  0.5006,  2.1007,  ..., -0.1256,  0.8334, -1.8840],
        [ 0.3221,  0.9576, -1.5949,  ...,  0.4771, -0.7206,  0.2753],
        [ 0.1482, -1.1207,  1.1867,  ...,  0.5207, -1.0125, -0.3823]],
       grad_fn=<EmbeddingBackward0>)


- 为了创建大语言模型（LLM）中使用的输入嵌入，我们只需将token嵌入和位置嵌入相加：

In [61]:
input_embeddings = token_embeddings + pos_embeddings#特征是词语信息跟位置信息的结合
print(input_embeddings.shape)

torch.Size([8, 4, 256])


- 在输入处理流程的初始阶段，输入文本被分割为独立的token。
- 在此分割后，这些token根据预定义的单词表转换为token ID：

<img src="../image/19.webp" width="400px">

# 总结与收获

请参见 [./dataloader.ipynb](./dataloader.ipynb) 代码笔记本，这是我们在本章中实现的数据加载器的简洁版，并将在后续章节中用于训练 GPT 模型。

请参见 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 获取习题解答。